In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import wrangle as w
import explore as exp

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve


In [15]:
# get_data function retrieves and caches the data in csv format ("nyc_collisions_{year}")
# subsequent runs will load the csv as to be respectful of the resources provided for free.
df = w.get_data(year_to_retrieve, app_token)
df
#for year 2022, retrieval took about 00:1:10 (mins) and 103,875 records

CSV file for 2022 already exists. Loading data from the CSV.


,crash_date,crash_time,latitude,longitude,location,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,off_street_name,vehicle_type_code2,borough,zip_code,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_4,vehicle_type_code_5
0,2022-01-01T00:00:00.000,7:33,40.771477,-73.918240,"{'latitude': '40.771477', 'longitude': '-73.91...",HOYT AVENUE NORTH,0,0,0,0,0,0,0,0,Unspecified,Unspecified,4491400,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:00:00.000,4:30,NaN,NaN,NaN,Southern parkway,0,0,0,0,0,0,0,0,Pavement Slippery,Unspecified,4491626,Sedan,Jfk expressway,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:00.000,7:57,NaN,NaN,NaN,WESTCHESTER AVENUE,0,0,0,0,0,0,0,0,Unspecified,NaN,4491734,Sedan,SHERIDAN EXPRESSWAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:00:00.000,5:17,40.746930,-73.848660,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,1,0,0,0,0,0,1,0,Passing or Lane Usage Improper,Unsafe Lane Changing,4491857,Sedan,NaN,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:00:00.000,1:30,40.819157,-73.960380,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,0,0,0,0,0,0,0,0,Unspecified,NaN,4491344,Sedan,NaN,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103870,2022-12-31T00:00:00.000,20:07,40.679165,-73.907510,"{'latitude': '40.679165', 'longitude': '-73.90...",NaN,1,0,1,0,0,0,0,0,Unspecified,NaN,4599890,Sedan,NaN,NaN,BROOKLYN,11233.0,102 SOMERS STREET,NaN,NaN,NaN,NaN,NaN,NaN
103871,2022-12-31T00:00:00.000,12:39,40.667210,-73.928505,"{'latitude': '40.66721', 'longitude': '-73.928...",NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4606651,Station Wagon/Sport Utility Vehicle,NaN,Sedan,BROOKLYN,11213.0,318 ROCHESTER AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
103872,2022-12-31T00:00:00.000,6:02,40.792225,-73.971985,"{'latitude': '40.792225', 'longitude': '-73.97...",NaN,0,0,0,0,0,0,0,0,Unspecified,NaN,4622905,Station Wagon/Sport Utility Vehicle,NaN,NaN,MANHATTAN,10025.0,201 WEST 93 STREET,NaN,NaN,NaN,NaN,NaN,NaN
103873,2022-12-31T00:00:00.000,18:25,40.851257,-73.935196,"{'latitude': '40.851257', 'longitude': '-73.93...",NaN,0,0,0,0,0,0,0,0,Passing Too Closely,Unspecified,4625305,Sedan,NaN,NaN,MANHATTAN,10033.0,4295 BROADWAY,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
duplicate_rows_df = df[df.duplicated(subset='collision_id', keep=False)]
duplicate_rows_df

,crash_date,crash_time,latitude,longitude,location,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,off_street_name,vehicle_type_code2,borough,zip_code,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_4,vehicle_type_code_5
68999,2022-08-30T00:00:00.000,18:40,40.635708,-73.954390,"{'latitude': '40.635708', 'longitude': '-73.95...",FARRAGUT ROAD,1,0,0,0,0,0,0,0,Failure to Keep Right,Driver Inattention/Distraction,4560055,Station Wagon/Sport Utility Vehicle,EAST 24 STREET,E-Bike,BROOKLYN,11210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69000,2022-08-30T00:00:00.000,18:40,40.635708,-73.954390,"{'latitude': '40.635708', 'longitude': '-73.95...",FARRAGUT ROAD,1,0,0,0,0,0,0,0,Failure to Keep Right,Driver Inattention/Distraction,4560055,Station Wagon/Sport Utility Vehicle,EAST 24 STREET,E-Bike,BROOKLYN,11210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70863,2022-09-06T00:00:00.000,19:00,40.756218,-73.857680,"{'latitude': '40.756218', 'longitude': '-73.85...",NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4562949,Sedan,NaN,NaN,QUEENS,11368.0,34-71 111 STREET,NaN,NaN,NaN,NaN,NaN,NaN
70864,2022-09-06T00:00:00.000,13:15,40.692310,-73.881470,"{'latitude': '40.69231', 'longitude': '-73.881...",JACKIE ROBINSON PKWY,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,4562362,Pick-up Truck,NaN,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71107,2022-09-06T00:00:00.000,19:00,40.756218,-73.857680,"{'latitude': '40.756218', 'longitude': '-73.85...",NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4562949,Sedan,NaN,NaN,QUEENS,11368.0,34-71 111 STREET,NaN,NaN,NaN,NaN,NaN,NaN
71127,2022-09-06T00:00:00.000,13:15,40.692310,-73.881470,"{'latitude': '40.69231', 'longitude': '-73.881...",JACKIE ROBINSON PKWY,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,4562362,Pick-up Truck,NaN,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71998,2022-09-09T00:00:00.000,20:49,40.808292,-73.948830,"{'latitude': '40.808292', 'longitude': '-73.94...",7 AVENUE,0,0,0,0,0,0,0,0,Unspecified,Unspecified,4563546,Station Wagon/Sport Utility Vehicle,WEST 124 STREET,NaN,MANHATTAN,10027.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71999,2022-09-09T00:00:00.000,18:24,40.849297,-73.853590,"{'latitude': '40.849297', 'longitude': '-73.85...",NaN,1,0,0,0,1,0,0,0,Unspecified,Unspecified,4563769,Sedan,NaN,Bike,BRONX,10461.0,1080 MORRIS PARK AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
72000,2022-09-09T00:00:00.000,20:49,40.808292,-73.948830,"{'latitude': '40.808292', 'longitude': '-73.94...",7 AVENUE,0,0,0,0,0,0,0,0,Unspecified,Unspecified,4563546,Station Wagon/Sport Utility Vehicle,WEST 124 STREET,NaN,MANHATTAN,10027.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72001,2022-09-09T00:00:00.000,18:24,40.849297,-73.853590,"{'latitude': '40.849297', 'longitude': '-73.85...",NaN,1,0,0,0,1,0,0,0,Unspecified,Unspecified,4563769,Sedan,NaN,Bike,BRONX,10461.0,1080 MORRIS PARK AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Remove duplicates in 'df' when all columns have the same information
df.drop_duplicates(keep='first', inplace=True)


In [ ]:
# Convert 'crash_date' and 'crash_time' to datetime and combine them into a single column
df['crash_datetime'] = pd.to_datetime(df['crash_date'].str[:10] + ' ' + df['crash_time'], format='%Y-%m-%d %H:%M')
df['crash_date'] = pd.to_datetime(df['crash_date'])


In [ ]:
df = w.initial_reorder_cols(df)

In [ ]:
# Create a mask where we filter out any observation with a value in vehicle 3 +
# This will leave us with only the observations that have 2 vehicles involved
condition = (df['vehicle_type_code_3'].isnull() &
             df['vehicle_type_code_4'].isnull() &
             df['vehicle_type_code_5'].isnull() &
             df['contributing_factor_vehicle_3'].isnull()
             )

# Apply the condition to filter the DataFrame
df = df[condition]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

df.drop(columns=['vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5', 'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5' ], inplace=True)


In [ ]:
# filter only keep observations with a value in vehicle_type_code1 and 2
# this will leave us with collisions of 2 vehicles only. 
condition = (df['vehicle_type_code1'].notnull() &
             df['vehicle_type_code2'].notnull())
df = df[condition]

# Reset the index if needed
df.reset_index(drop=True, inplace=True)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# Filter the DataFrame to exclude collisions involving pedestrians or cyclists
df = df[(df['number_of_pedestrians_injured'] == 0) & (df['number_of_pedestrians_killed'] == 0) & (df['number_of_cyclist_injured'] == 0) & (df['number_of_cyclist_killed'] == 0)]

In [ ]:
df = df.drop(columns=['cross_street_name', 'off_street_name', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed'])

In [ ]:
# Create a boolean mask for rows where injuries do not match
injury_mask = df['number_of_persons_injured'] != df['number_of_motorist_injured']

# Create a boolean mask for rows where there are no injuries or deaths involving pedestrians or cyclists
no_pedestrian_cyclist_injuries_mask = (
    (df['number_of_pedestrians_injured'] == 0) &
    (df['number_of_pedestrians_killed'] == 0) &
    (df['number_of_cyclist_injured'] == 0) &
    (df['number_of_cyclist_killed'] == 0)
)

# Combine the two masks using the & operator to get the final filtered DataFrame
filtered_df = df[injury_mask & no_pedestrian_cyclist_injuries_mask]

filtered_df

In [ ]:
len(filtered_df)

In [ ]:
# Create a boolean mask for rows where injuries do not match
mask = df['number_of_persons_injured'] != df['number_of_motorist_injured']

# Use the mask to filter the DataFrame and get rows where injuries do not match
non_matching_injuries = df[mask]

# To display rows where injuries do not match, you can use:
non_matching_injuries


In [ ]:
# Create a filter for rows with duplicate location coordinates
duplicate_location_filter = df['location'].duplicated(keep=False)

# Create a filter for rows with missing values in on_street_name, borough, or zip_code
missing_info_filter = df['on_street_name'].isnull() | df['borough'].isnull() | df['zip_code'].isnull()

# Create a filter for rows with latitude not equal to 0.0
non_zero_latitude_filter = df['latitude'] != 0.0

# Combine the filters to get rows that meet all conditions
result = df[duplicate_location_filter & missing_info_filter & non_zero_latitude_filter]

# Now, 'result' contains rows where the location coordinates already exist in a different row of the DataFrame based on the 'location' column,
# they also have missing values in on_street_name, borough, or zip_code, and latitude is not equal to 0.0.

# Sort the DataFrame by the 'location' column
sorted_df = result.sort_values(by='location', ascending=True)

sorted_df.head()


In [ ]:
txt = sorted_df.iloc[0]['location']


In [ ]:
count = len(df[(df['latitude'] == 0) & (df['location'] != txt)])
count

In [ ]:
# Replace 'latitude' with the actual column name for latitude
# Replace 'longitude' with the actual column name for longitude
# Replace 'location' with the actual column name for location

df.loc[df['latitude'] == 0.0, ['latitude', 'longitude', 'location']] = [np.nan, np.nan, np.nan]

In [ ]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

____
After cleaning some of the data, I decided to go and use google api to fill in missing geo location information. I will now load those changes and continue cleaning the dataframe
___

In [ ]:
df = w.wrangle_coll_stage1(year_to_retrieve, app_token)
df.head()

In [ ]:
len(df)

In [ ]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

In [ ]:
df.contributing_factor_vehicle_2.unique()

In [ ]:
# Assuming you have a DataFrame named 'df'
df['contributing_factor_vehicle_2'] = df['contributing_factor_vehicle_2'].fillna('Unspecified')

In [ ]:
df.contributing_factor_vehicle_1.unique()

In [ ]:
# Assuming you have a DataFrame named 'df'
df['contributing_factor_vehicle_1'] = df['contributing_factor_vehicle_1'].fillna('Unspecified')

In [ ]:
df.dropna(subset=['latitude', 'longitude'], inplace=True)


In [ ]:
len(df)

In [ ]:
# Create 'injuries_count' column based on the higher of the two injury columns
df['injuries_count'] = df[['number_of_persons_injured', 'number_of_motorist_injured']].max(axis=1)

# Create 'deaths_count' column based on the higher of the two death columns
df['deaths_count'] = df[['number_of_persons_killed', 'number_of_motorist_killed']].max(axis=1)

# Create 'injuries' column based on 'injuries_count'
df['injuries'] = df['injuries_count'] > 0

# Create 'deaths' column based on 'deaths_count'
df['deaths'] = df['deaths_count'] > 0

In [ ]:
df =  df.drop(['number_of_persons_injured', 'number_of_motorist_injured', 'number_of_persons_killed', 'number_of_motorist_killed'], axis=1)

In [ ]:
df.injuries.unique()

In [ ]:
df.head()

In [4]:
df = w.wrangle_coll_stage1(year_to_retrieve, app_token)
df.head()

CSV file for 2022 already exists. Loading data from the CSV.


,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,on_street_name,borough,zip_code,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,injuries_count,deaths_count,injuries,deaths
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.851690,-73.952380,CROSS BRONX EXPY,NaN,7024,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified,0,0,False,False
1,2022-11-17 10:50:00,2022-11-17,10:50,4582618,40.802204,-73.967804,BROADWAY,Manhattan,10025,Station Wagon/Sport Utility Vehicle,Passing or Lane Usage Improper,Box Truck,Passing or Lane Usage Improper,0,0,False,False
2,2022-11-17 14:40:00,2022-11-17,14:40,4582613,40.756664,-73.968860,East 52nd Street,MANHATTAN,10022,Taxi,Passing Too Closely,Flat Bed,Unspecified,0,0,False,False
3,2022-11-17 15:00:00,2022-11-17,15:00,4583338,40.865364,-73.901344,Grand Avenue,BRONX,10468,Sedan,Driver Inattention/Distraction,Bus,Unspecified,0,0,False,False
4,2022-11-17 14:36:00,2022-11-17,14:36,4583745,40.714336,-73.946370,Metropolitan Avenue,BROOKLYN,11211,Box Truck,Following Too Closely,Sedan,Unspecified,0,0,False,False


In [5]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,21168,44.0,0,0,24571,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,48254,int64
4,latitude,0,0.0,0,0,23107,float64
5,longitude,0,0.0,0,0,19572,float64
6,on_street_name,0,0.0,0,0,5600,object
7,borough,1,0.0,0,0,10,object
8,zip_code,0,0.0,0,0,211,int64
9,vehicle_type_code1,0,0.0,0,0,245,object


In [11]:
df_with_nulls = df[df['crash_datetime'].notnull()]
df_with_nulls

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,on_street_name,borough,zip_code,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2,injuries_count,deaths_count,injuries,deaths
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.851690,-73.952380,CROSS BRONX EXPY,NaN,7024,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified,0,0,False,False
1,2022-11-17 10:50:00,2022-11-17,10:50,4582618,40.802204,-73.967804,BROADWAY,Manhattan,10025,Station Wagon/Sport Utility Vehicle,Passing or Lane Usage Improper,Box Truck,Passing or Lane Usage Improper,0,0,False,False
2,2022-11-17 14:40:00,2022-11-17,14:40,4582613,40.756664,-73.968860,East 52nd Street,MANHATTAN,10022,Taxi,Passing Too Closely,Flat Bed,Unspecified,0,0,False,False
3,2022-11-17 15:00:00,2022-11-17,15:00,4583338,40.865364,-73.901344,Grand Avenue,BRONX,10468,Sedan,Driver Inattention/Distraction,Bus,Unspecified,0,0,False,False
4,2022-11-17 14:36:00,2022-11-17,14:36,4583745,40.714336,-73.946370,Metropolitan Avenue,BROOKLYN,11211,Box Truck,Following Too Closely,Sedan,Unspecified,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27090,2022-11-17 12:00:00,2022-11-17,12:00,4583142,40.732445,-74.005470,BEDFORD STREET,Manhattan,10014,Station Wagon/Sport Utility Vehicle,Other Vehicular,Moped,Other Vehicular,1,0,True,False
27091,2022-11-17 08:50:00,2022-11-17,8:50,4582518,40.673570,-73.952940,PARK PLACE,Brooklyn,11216,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified,0,0,False,False
27092,2022-11-17 14:30:00,2022-11-17,14:30,4582575,40.603348,-74.120700,Manor Road,STATEN ISLAND,10314,Station Wagon/Sport Utility Vehicle,Following Too Closely,Moped,Unspecified,1,0,True,False
27093,2022-11-17 11:35:00,2022-11-17,11:35,4582544,40.738434,-73.993450,West 17th Street,MANHATTAN,10011,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified,0,0,False,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48263 entries, 0 to 54213
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 27095 non-null  datetime64[ns]
 1   crash_date                     48263 non-null  datetime64[ns]
 2   crash_time                     48263 non-null  object        
 3   collision_id                   48263 non-null  int64         
 4   latitude                       48263 non-null  float64       
 5   longitude                      48263 non-null  float64       
 6   on_street_name                 48263 non-null  object        
 7   borough                        48262 non-null  object        
 8   zip_code                       48263 non-null  int64         
 9   vehicle_type_code1             48263 non-null  object        
 10  contributing_factor_vehicle_1  48263 non-null  object        
 11  vehicle_type_code2  

In [10]:
df['zip_code'] = df['zip_code'].astype('int64')

In [5]:
df = w.get_data(2022, app_token, max_req)


CSV file for 2022 already exists. Loading data from the CSV.


In [ ]:
#df = date_time(df)


In [7]:
df = w.initial_reorder_cols(df)


In [9]:
df = w.drop_3plus(df)


In [11]:
df = w.single_coll(df)


In [13]:
df = w.motorist_only(df)


In [15]:
df = w.second_drop(df)


In [16]:
df = w.null_zero_loc(df)


In [18]:
df = w.fill_geo(df)


In [ ]:
df = fill_na(df)


In [ ]:
df = drop_na(df)


In [ ]:
df = injuries_build(df)

In [ ]:
df = single_coll(df)
df = motorist_only(df)
df = second_drop(df)
df = null_zero_loc(df)
df = fill_geo(df)
df = fill_na(df)
df = drop_na(df)
df = injuries_build(df)